In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
import googlemaps
from geopy import distance
import folium

In [ ]:
data = pd.read_csv('lista_final.csv',sep='|')

In [ ]:
load_dotenv()

In [ ]:
API_key=os.getenv('API')

In [ ]:
gmaps_key = googlemaps.Client(key=API_key)

In [ ]:
def get_coord(string):
    geocode_result = gmaps_key.geocode(string)
    try:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
        return [lat,lon]
    except:
        return [None,None]

In [ ]:
#data['Rango precios'] = data['Price(MX$)']

In [ ]:
#data.info()

In [ ]:
def rentas():
    rango_precios = {'1':'5000-10000','2':'10000-15000','3':'15000-20000','4':'20000-25000','5':'25000-30000','6':'30000-35000','7':'35000-40000','8':'40000-45000','9':'45000-50000','10':'>50000'}
    dummy = data.copy()
    
    indice_precios = rango_precios[input(f'Selecciona un rango de precios: {str(rango_precios)}')].split('-')
    
    dummy = dummy[dummy['Price(MX$)'].between(int(indice_precios[0]), int(indice_precios[1]))]
    
    indice_hab = int(input((f'Selecciona número de habitantes: ')))
    dummy = dummy[dummy['Bedrooms'] == indice_hab]

    coord_trab = input(f'Cual es tu ubicacion? : ')

    ubicacion = get_coord(coord_trab)
    dummy['Distance'] = dummy['Coordinates'].str.split(',').apply(\
                        lambda x:get_distance(x, ubicacion))
    
    dummy = dummy[dummy['Distance']<=5]
    
    return dummy, ubicacion

In [ ]:
test, ubi = rentas()

In [ ]:
ubi

In [ ]:
test['Address']

In [ ]:
depas = rentas()

In [ ]:
depas

In [ ]:
#depas.sort_value(by='Distance').head()

In [ ]:
def get_distance(pointx, pointy):
    return distance.distance(pointx,pointy)

In [ ]:
get_distance([-39, 99], [-40, 98])

In [ ]:
#Calcular coordenadas
data['Distance'] = data['Coordinates'].apply(lambda x: get_distance(x.split(",")))

In [ ]:
#data.to_csv('Final_distances.csv', sep="|", index=False)

In [ ]:
mapa = folium.Map(
    location=depas['Lugar de trabajo'],
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'
mapa

In [ ]:
for index, row in data.head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row['Address'],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa)

In [ ]:
mapa

In [1]:
mapa2 = folium.Map(
    location=depas['Lugar de trabajo'],
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'


for index, row in data.sort_values(by='Distance').head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=15,
                  popup=row[['Address','Bedrooms','Price(MX$)']],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa2)


NameError: name 'folium' is not defined

In [ ]:
ubi

In [ ]:
mapa2 = folium.Map(
    location=depas['Lugar de trabajo'],
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'


for index, row in test.sort_values(by='Distance').head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row[['Address','Bedrooms','Price(MX$)']],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa2)
    
folium.Marker(ubi, icon=folium.Icon(color='blue',icon='home')).add_to(mapa2)


In [ ]:
mapa2